In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from collections import deque
from collections import defaultdict
import random

# 1. Data

In [ ]:
df_hero_net = pd.read_csv(r"C:\Users\Marina\OneDrive\Desktop\archive\hero-network.csv")
df_edges = pd.read_csv(r"C:\Users\Marina\OneDrive\Desktop\archive\edges.csv")
df_nodes = pd.read_csv(r"C:\Users\Marina\OneDrive\Desktop\archive\nodes.csv")

In [ ]:
df_hero_net

In [ ]:
df_edges

In [ ]:
df_nodes

In [ ]:
df_nodes.shape

In [ ]:
n_comics = df_nodes[df_nodes.type == 'comic'].count()
n_heros = df_nodes[df_nodes.type == 'hero'].count()
print(n_comics, n_heros)

## 1.1 Pre-processing 

In [ ]:
We remove extra spaces and '/' at the end of the names in the datasets. 

In [ ]:
df_hero_net["hero1"] = df_hero_net["hero1"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)
df_hero_net["hero2"] = df_hero_net["hero2"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)

In [ ]:
df_hero_net

In [ ]:
df_edges["hero"] = df_edges["hero"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)
df_edges["comic"] = df_edges["comic"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)

In [ ]:
df_edges

In [ ]:
df_nodes["node"] = df_nodes["node"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)

In [ ]:
df_nodes

 We fix the 'spider-man/peter parker' error in the datasets.

In [ ]:
df_hero_net = df_hero_net.replace('SPIDER-MAN/PETER PAR','SPIDER-MAN/PETER PARKER', regex=True)

In [ ]:
df_nodes.node = df_nodes.node.replace('SPIDER-MAN/PETER PARKERKER','SPIDER-MAN/PETER PARKER', regex=True)

We check if the error is fixed and if there are other similar errors in the datasets.

At first, we check the hero names.

In [ ]:
hero_edges = set(df_edges.hero)
hero_heronet = set(df_hero_net.hero1).union(set(df_hero_net.hero2))
hero_nodes = set(df_nodes.loc[df_nodes.type=='hero']['node'])

print(hero_edges - hero_heronet)
print(hero_heronet - hero_edges)

In [ ]:
print(hero_edges - hero_nodes)
print(hero_nodes - hero_edges)

In [ ]:
print(hero_nodes - hero_heronet)
print(hero_heronet - hero_nodes)

Then, we check the comic names.

In [ ]:
comic_edges = set(df_edges.comic)
comic_nodes = set(df_nodes.loc[df_nodes.type=='comic']['node'])

In [ ]:
print(comic_edges-comic_nodes)
print(comic_nodes-comic_edges)

We notice that some hero names are the same as comic names. We therefore decide to add word 'comic' at the end to the comic names equal to those of the heroes, in order to avoid problems in the creation and management of graphs.

In [ ]:
equals = hero_edges.intersection(comic_edges)
equals

In [ ]:
hero_nodes.intersection(comic_nodes)

In [ ]:
for i in equals:
    df_edges.comic = df_edges.comic.replace(i, i+' comic', regex=True)


## 1.2 First graph

In [ ]:
G1 = nx.MultiGraph()

In [ ]:
df_hero_net.apply(lambda row: G1.add_edge(row['hero1'], row['hero2']), axis=1)

In [ ]:
#drop sefl-loops
G1.remove_edges_from(nx.selfloop_edges(G1))

In [ ]:
list(G1.edges())[:10]

In [ ]:
dicOfOccurances = {}
for h1,h2 in df_hero_net.to_numpy():
    if h1 != h2:
        if (h1,h2) in dicOfOccurances:
            dicOfOccurances[(h1,h2)] += 1
        else:
            dicOfOccurances[(h1,h2)] = 1


In [ ]:
G1_weighted = nx.MultiGraph()

In [ ]:
i = 0
for edge in list(G1.edges()):
    try:
        G1_weighted.add_edge(edge[0], edge[1], weight = round(1/dicOfOccurances[edge], 3))
    except:
        G1_weighted.add_edge(edge[0], edge[1], weight = round(1/dicOfOccurances[(edge[1], edge[0])],3))


In [ ]:
edges, weights = zip(*nx.get_edge_attributes(G1_weighted, 'weight').items())
for i in range(20):
    print((edges[i], weights[i]))

In [ ]:
G1_weighted['LITTLE, ABNER']['PRINCESS ZANDA']

In [ ]:
G1_weighted_graph = nx.Graph(G1_weighted)
G1_weighted_graph['LITTLE, ABNER']['PRINCESS ZANDA']

In [ ]:
edges, weights = zip(*nx.get_edge_attributes(G1_weighted_graph, 'weight').items())
for i in range(20):
    print((edges[i], weights[i]))

In [ ]:
dicOfWeights = {}
for edge in G1_weighted_graph.edges():
    dicOfWeights[edge] = G1_weighted_graph.get_edge_data(edge[0], edge[1])['weight']


In [ ]:
print('Number of nodes of the first graph: ' + str(len(G1_weighted_graph.nodes())))
print('Number of edges of the first graph: ' +  str(len(G1_weighted_graph.edges())))

## 1.3 Second graph

In [ ]:
attributes = {}
for elem in df_edges['hero']:
    attributes[elem] = {'type' : 'hero' } 
for elem in df_edges['comic']:
    attributes[elem] = {'type' : 'comic' } 

print('Number of elements in attributes list: ' + str(len(attributes)))

In [ ]:
G2 = nx.from_pandas_edgelist(df_edges, 'hero', 'comic')
nx.set_node_attributes(G2, attributes)
print('Number of nodes of the second graph: ' + str(len(G2.nodes())))
print('Number of edges of the second graph: ' +  str(len(G2.edges())))

In [ ]:
nodes, types = zip(*nx.get_node_attributes(G2, 'type').items())
for i in range(10):
    print(nodes[i], types[i])

In [ ]:
def top_N(df, N):
    return df.groupby("hero", group_keys=False).count().sort_values(by='comic', ascending = False).head(N)
def create_G1_topN(G, N, df):
    G_topN = G.copy()
    most_popular_heros = top_N(df, N)
    notInTopN = set(G.nodes()) - set(most_popular_heros.index)
    G_topN.remove_nodes_from(notInTopN)
    return G_topN


# Functionality 1

In [ ]:
def degree_distG(G):
    degree_G = [G.degree(n) for n in G.nodes()]
    plt.hist(degree_G, bins=100, color='green', edgecolor='black')
    plt.xlim(0, 1500)
    plt.title('Degree Distribution of G1')
    plt.ylabel('Number of nodes')
    plt.xlabel('Degree')
    plt.show()

In [ ]:
def hubsG(G):
    degreeG1 = [G.degree(n) for n in G.nodes()]
    ex_nodes = [n for n in G.nodes() if G.degree(n) > np.percentile(degreeG1,95)]
    return(ex_nodes)

In [ ]:
def hubs(G):
    degreeG2 = [G.degree(n) for n in G.nodes()]
    ex_nodes = [n for n in G.nodes() if G.degree(n) > np.percentile(degreeG2,95) and G.nodes[n].get('type', '') == 'comic']
    return(ex_nodes)

In [ ]:
def density_G(G):
    n = G.number_of_nodes()
    m = G.number_of_edges()
    return 2*m/(n*(n-1))

In [ ]:
def functionality_1(G,G_type,N):
    G = G.copy()
    if G_type == 1:
        G = create_G1_topN(G,n,df_edges)
        
        n_nodesG1 = int(G.number_of_nodes())
        degree_G1 = G.degree()
        
        sorted_degree_G1 = sorted(degree_G1, key=lambda x: x[1], reverse=True)
    
        densityG1 = len(G.edges())/(n_nodesG1*(n_nodesG1-1)/2)
        
        degree_sequence = [deg for i, deg in G.degree()]
        avg_degreeG1 = sum(degree_sequence) / len(degree_sequence)
        
        type_graphG1 = ""    
        if density_G(G) >= 0.5:
            type_graphG1 = "DENSE"
        else:
            type_graphG1 = "SPARSE"
            
    
        print("Number of nodes of G1: ", n_nodesG1)
        print("sorted collaborations of each node:", sorted_degree_G1)
        print("The density of G1 is:", densityG1)
        print(degree_distG(G))
        print("Average degree of G1: ", avg_degreeG1)
        print("Nodes with degrees more extensive than the 95th percentile of the degree distribution: ", len(hubs(G)), 
        "\n", sorted(hubsG1(G), key=lambda x: G.degree(x), reverse=True))
        print("The type of G1 is: ", type_graphG1)
    
    
    
    if G_type == 2:
        
        
        count_hero = 0
        count_comic = 0
        for elements in types:
            if elements == 'hero':
                count_hero += 1
            elif elements == 'comic':
                count_comic += 1
        degree_G = G.degree()
        
        hero_appeard = df_edges.groupby('comic').count().sort_values(by='hero', ascending=False)
    
        densityG2 = len(G.edges())/(count_hero*(count_hero-1)/2)
        
        degree_sequence = [deg for i, deg in G.degree()]
        avg_degreeG2 = sum(degree_sequence) / len(degree_sequence)
        
        type_graphG2 = ""    
        if density_G(G) >= 0.5:
            type_graphG2 = "DENSE"
        else:
            type_graphG2 = "SPARSE"
    
    
        print("Number of nodes of G2: ", count_hero, count_comic)
        print( "the number of heroes that have appeared in each comic of G2:" , "\n", hero_appeard)
        print("The density of G1 is:", densityG2)
        print(degree_distG(G))
        print("Average degree of G2: ", avg_degreeG2)
        print("Nodes with degrees more extensive than the 95th percentile of the degree distribution: ", len(hubs(G)),
         "\n", sorted(hubs(G), key=lambda x: G.degree(x), reverse=True))    
        print("The type of G2 is: ", type_graphG2)

     
    
    
    return 

# Functionality 2 - Find top superheroes!

In [ ]:
def fun_betweenness_centrality(G):
    betweenness = defaultdict(int)
    for i, s in enumerate(G):
        if i == 10:
            break
        for j, t in enumerate(G):
            if j == 10:
                break
          if s != t:
            # Calcolo i percorsi minimi tra s e t utilizzando l'algoritmo BFS
            path = nx_shortest_path(G,source=s, target=t)
            for v in path:
              betweenness[v] += 1
      # Normalizza i valori della betweenness
      n = len(G)
      for v in betweenness:
        betweenness[v] /= (n-1)*(n-2)
      return betweenness

In [ ]:
def fun_pagerank_centrality(G, alpha=0.85, max_iter=100, tol=1e-6):
  # Inizializza i valori di PageRank di ogni nodo con 1/n, dove n è il numero di nodi del grafo
  n = len(G)
  pagerank = {node: 1/n for node in G}
  # Itera finché non si raggiunge la convergenza o il numero massimo di iterazioni
  for i in range(max_iter):
    diff = 0
    # Calcola il nuovo valore di PageRank per ogni nodo
    for node in pagerank:
      rank = sum(pagerank[neighbor] / len(G[neighbor]) for neighbor in G[node])
      # Aggiorna il valore di PageRank del nodo utilizzando la formula di PageRank
      new_rank = (1 - alpha) / n + alpha * rank
      diff += abs(new_rank - pagerank[node])
      pagerank[node] = new_rank
    # Se la differenza è minore della soglia di tolleranza, interrompi il loop
    if diff < tol:
      break
  return pagerank

In [ ]:
def fun_degree_centrality(G, node):
    num_neighbors = len(list(G.neighbors(node)))
    fun_degree_centrality = num_neighbors / (len(G) - 1)
    return fun_degree_centrality

In [ ]:
def fun_closeness_centrality(G, node):
  # Inizializza una coda e un dizionario per tenere traccia della distanza di ogni nodo dal nodo in questione
  queue = deque([node])
  distances = {node: 0}

  # Esegue la ricerca in larghezza a partire dal nodo in questione
  while queue:
    current_node = queue.popleft()
    for neighbor in G[current_node]:
      if neighbor not in distances:
        distances[neighbor] = distances[current_node] + 1
        queue.append(neighbor)

  # Calcola la somma delle distanze di tutti i nodi dal nodo in questione
  total_distance = sum(distances.values())
  # Calcola la centralità di prossimità come il numero di nodi del grafo diviso dalla somma delle distanze
  closeness_centrality = len(G) / total_distance
  return closeness_centrality

In [ ]:
def functionality_2(G,node,metric,N):
    
    G1_topN = create_G1_topN(G,n,df_edges)

    result = fun_betweenness_centrality(G)
    
    for nodo in G.nodes() and N:
        print("the node is: ", nodo)
    if G == G1_topN:
        print(" the betweenness centrality of G1: ", fun_betweenness_centrality(G1))
        print(" the degree centrality of G1: ", fun_degree_centrality(G1, nodo))
        print(" the pagerank centrality of G1: ", fun_pagerank_centrality(G1))
        print(" the closeness centrality of G1: ", fun_closeness_centrality(G1, top_n_heroes))
    else:
        print(" the betweenness centrality of G2: ", fun_betweenness_centrality(G2))
        print(" the degree centrality of G2: ", fun_degree_centrality(G2, nodo))
        print(" the pagerank centrality of G2: ", fun_pagerank_centrality(G2))
        print(" the closeness centrality of G2: ", fun_closeness_centrality(G2, topN_heroes))

    
    
    

In [ ]:
pagerank = nx.pagerank(G)
    count = 0
    for node, value in pagerank.items():
        print(f"{node}: {value}")
        count += 1
        if count == 10:
            break

## Functionality 3

In [ ]:
def top_N(df,N,G):
    df_new = df.groupby(['hero'])['hero'].count().reset_index(name="count")
    df_new= df_new.sort_values(by = 'count',ascending=False)
    df_new = df_new.head(N)
    nodes=[df_new['hero'][i] for i in df_new.index]
    
    new_nodes = set(nodes)
    for i in nodes:
        app = list(df_edges.loc[df_edges['hero']==i]['comic'])
        new_nodes.update(app)
        
    G_top_N = G.subgraph(new_nodes)
    return G_top_N

In [ ]:
def shortest_path_two_nodes(node_1, node_2, G, list_nodes):
    if not nx.has_path(G,node_1,node_2):
        return []
    new_list = set(list_nodes)
    new_list.remove(node_1)
    new_list.remove(node_2)
    nodes = [i for i in G.nodes if i not in new_list]
    return list(nx.shortest_path(G.subgraph(nodes), source=node_1, target=node_2, weight=None))

In [ ]:
def functionality_3(G2, N, h_1, h_n, h):
    if N:
        N = int(N)
        G2 = top_N(df_edges,N,G2)
    att = nx.get_node_attributes(G2, "type")
    if not nx.is_connected(G2):
        result = 'There is not such path because the graph is not connected'
        return result
    if h_1  not in list(G2.nodes) or h_n not in list(G2.nodes):
        result = 'There is not such path'
        return result
    shortest_path = np.array([h_1])
    s = h_1
    for t in h:
        app = shortest_path_two_nodes(s,t,G2,[h_1]+h+[h_n])
        if app == []:
            return [], G2
        #app = list(nx.shortest_path(G2, source=s, target=t, weight=None))
        s = t
        app = np.array(app)
        shortest_path = np.concatenate((shortest_path,app[1:]),axis=None)
    app = list(nx.shortest_path(G2, source=t, target=h_n, weight=None))
    app = np.array(app)
    shortest_path = np.concatenate((shortest_path,app[1:]),axis=None)
    
    return shortest_path, G2

# Functionality 4

In [ ]:
def search_shortest_path(G, source, target):
    try:
        return nx.shortest_path(G, source, target, weight='weight', method='dijkstra')
    except: 
        return 0

In [ ]:
def functionality_4(G, heroA, heroB, N, df):
    count = 0
    G_final = create_G_topN(G, N, df)

    while search_shortest_path(G_final, heroA, heroB) != 0:
        shortest_path = search_shortest_path(G_final, heroA, heroB)
        dic = {}
        for i in range(len(shortest_path)-1):
            weight = G.get_edge_data(shortest_path[i], shortest_path[i+1])['weight']
            dic[(shortest_path[i],shortest_path[i+1])] = weight
        
        dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse = True))       
        G_final.remove_edge(list(dic.keys())[0][0], list(dic.keys())[0][1])
        count+=1
    
    colors = ['red' if node == heroA or node == heroB else 'blue' 
                for node in G_final.nodes()]
                
    return G_final, count, colors

# Functionality 5

In [ ]:
def top_N_1(df,N,G):
    df_new = df.groupby(['hero'])['hero'].count().reset_index(name="count")
    df_new= df_new.sort_values(by = 'count',ascending=False)
    df_new = df_new.head(N)
    nodes=[df_new['hero'][i] for i in df_new.index]
    G_top_N = G.subgraph(nodes)
    return G_top_N

In [ ]:
def edges_to_remove(G):
    
    num_components = nx.number_connected_components(G)
    
    edges_to_remove = 0
    while True:
        if num_components > 1:
                break
        G_dict = {}
        G_dict = nx.edge_betweenness_centrality(G)
        max_EBC = max(G_dict.values())
        edges = []
        for key, value in sorted(G_dict.items(), key=lambda item: item[1], reverse = True):
            if value < max_EBC:
                break
            edges.append(key)
        edges_to_remove += len(edge)
        for e in edges:
            G.remove_edge(e[0], e[1])
        components = nx.connected_components(G)
        num_components = nx.number_connected_components(G)
    return edges_to_remove, G

In [ ]:
def functionality_5(G1,N, Hero_1, Hero_2):
    G_top_N = top_N_1(df_edges,N,G1)
    G = nx.Graph(G_top_N)
    
    hero_1_2_together = False
    number_of_edges, G = edges_to_remove(G)
    comunities = []
    for comunity in nx.connected_components(G):
        comunities.append(list(comunity))
        if Hero_1 in comunity and Hero_2 in comunity:
            hero_1_2_together = True
    

    return number_of_edges, G, comunities, hero_1_2_together, G_top_N

# 3. Frontend Implementationc

## Visualization 1 - Visualize some features of the network